# a-5. 結合
２つ以上のDataFrameを一つのDataFrameに結合する処理を扱う。
<b>merge()</b>と<b>join()</b>は列を結合する処理で、一般的に<b>結合キー</b>と呼ばれる列を突き合わせながら２つの以上の表を１つに結合する。
DataFrame <b>L</b>にDataFrame <b>R</b>を結合することを考えると、
<b>R</b>の結合キーとしてindexを用いる結合がjoin()メソッドで、<b>L</b>,<b>R</b>両方についてより汎用的な結合キーを指定できるのがmerge()メソッドである。
merge()は列の結合についての汎用的な機能を提供し、ある特化した（しかし一般的によく利用される)シーンで用いるのがjoin()と考えればよい。
join()の典型的な利用シーンは、1-2の図に示されるように、売上データのようなトランザクションデータに商品マスターなどのマスターデータを結合する例を考えるとわかりやすい。
そして<b>concat()</b>メソッドは、列と行のラベル(indexとcolumn)によって2つ以上のDataFrameを結合する。join()やmerge()のように列を結合キーにすることはできない。
結合対象の複数のDataFrameに重複するラベルがあればjoin()のように機能する。
ただし、経験上、よく利用するケースは、同じ列名で構成される複数のDataFrameを縦に連結するといった操作である。


---
## 1. 結合の代表例
## 2.列の結合(merge, join)
### &nbsp; 2-1. 結合キーの重複(1:1, m:1, m:m)
### &nbsp; 2-2. 結合キーの不一致(inner, left, right, outer)
### &nbsp; 2-3. Seriesの結合

## 3.行と列の連結(concat)

### 関連ドキュメント
- [Merge, join, concatenate and compare](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

### APIリファレンス
- [DataFrame.join](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html)
- [DataFrame.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)
- [pandas.merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)
- [pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
---

In [1]:
import numpy as np
import pandas as pd

# 1. 結合の代表例(merge, join, concat)

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
tra = pd.DataFrame(
    {'customer':['A', 'A', 'A', 'B', 'C', 'C', 'C'],
     'item':['coffee', 'bread', 'milk', 'juice', 'tea', 'milk', 'bread']}
)

cmaster = pd.DataFrame(
    {'customer':['A', 'C'],
     'age':[40, 25],
     'gender':['female', 'male']}
)

imaster = pd.DataFrame(
    {'item':['coffee', 'bread', 'milk', 'tea'],
     'price':[100, 120, 110, 80]}
)

imaster = imaster.set_index('item')
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:45%;">
      <col style="width:22%;">
    </colgroup>
<tbody>
 
<tr><th align="left">no.</th><th align="left">実行コード</th><th align="left">入力 / 出力</th><th>備考</th></tr>

<tr><td colspan=4 bgcolor="#8fbc8f"><b>merge()</b></td></tr>
   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(tra.merge(cmaster, on='customer'))
</pre></td>
<td><img width="80%" src="./figures/join/join1-1.png"></td>
<td>
商品購入データのcustomer列と顧客マスター(cmaster)のcustomer列を結合キーにして、年齢(age)と性別(gender)列をを結合する。
merge()は、このように結合される側(leftと呼ぶ)も、結合する側(rightと呼ぶ)も、結合キーに列名を指定することが多い。
ただし、merge()は多様な結合を実現できるように設計されており、結合キーがleft/right問わずにindexであっても結合可能である(次の例を参照)。
</td>
</tr>
 
<tr><td colspan=4 bgcolor="#8fbc8f"><b>join()</b></td></tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(tra.join(imaster, on='item'))
# 以下のようにmerge()を使っても同じ結果を得られるがパラメータの指定が複雑となる。
print(tra.merge(imaster, left_on='item', right_index=True, how='left'))
</pre></td>
<td><img width="80%" src="./figures/join/join1-2.png"></td>
<td>
商品購入データのitem列を結合キーにして、imasterのindexを突き合わせて商品(item)列を結合する。
join()は、このように結合される側(leftと呼ぶ)の結合キーは列で、結合する側(rightと呼ぶ)がindexであるような結合となる場合が多い。
また、２つめの実行例のように、join()で実現できることはmergeでも実現可能であるが、パラメータの指定方法が少し複雑になる。
mergeで指定したパラメータについては後述している。
</td>
</tr>
   
<tr><td colspan=4 bgcolor="#8fbc8f"><b>concat()</b></td></tr>
   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([cmaster, cmaster]))
</pre></td>
<td><img width="90%" src="./figures/join/join1-3.png"></td>
<td>
顧客マスターを縦に連結する例。
このように同じ列で構成される複数のDataFrameを縦に連結するケースが多いが、より複雑な連結も実現できる。詳細は後述する。
連結された行indexは元のindexがそのまま採用されるため、indexが重複していることに注意する。
</td>
</tr>
    
</tbody>
</table>

In [2]:
print('''
# 入力データ
# traは顧客がどの商品を購入したかに関するトランザクションデータである。
# custは顧客の年齢と性別を示した顧客マスターデータである。
# 一方でitemは商品の値段を示した商品マスターでcustomer列は行indexとしてセットしている。
>>> tra = pd.DataFrame(
        {'customer':['A', 'A', 'A', 'B', 'C', 'C', 'C'],
         'item':['coffee', 'bread', 'milk', 'juice', 'tea', 'milk', 'bread']}
    )
>>> print(tra)''')

tra = pd.DataFrame(
    {'customer':['A', 'A', 'A', 'B', 'C', 'C', 'C'],
     'item':['coffee', 'bread', 'milk', 'juice', 'tea', 'milk', 'bread']}
)
print(tra)

print('''
>>> cmaster = pd.DataFrame(
        {'customer':['A', 'C'],
         'age':[40, 25],
         'gender':['female', 'male']}
    )
>>> print(cmaster)''')
cmaster = pd.DataFrame(
    {'customer':['A', 'C'],
     'age':[40, 25],
     'gender':['female', 'male']}
)
print(cmaster)

print('''
>>> imaster = pd.DataFrame(
        {'item':['coffee', 'bread', 'milk', 'tea'],
         'price':[100, 120, 110, 80]}
     )
>>> imaster = imaster.set_index('item')
>>> print(imaster)''')
imaster = pd.DataFrame(
    {'item':['coffee', 'bread', 'milk', 'tea'],
     'price':[100, 120, 110, 80]}
)
imaster = imaster.set_index('item')
print(imaster)

print('''
## merge
# no.1-1
# 商品購入データのcustomer列と顧客マスター(cmaster)のcustomer列を結合キーにして、年齢(age)と性別(gender)列をを結合する。
# merge()は、このように結合される側(leftと呼ぶ)も、結合する側(rightと呼ぶ)も、結合キーに列名を指定することが多い。
# ただし、merge()は多様な結合を実現できるように設計されており、結合キーがleft/right問わずにindexであっても結合可能である(次の例を参照)。
>>> print(tra.merge(cmaster, on='customer'))''')
print(tra.merge(cmaster, on='customer'))

print('''
## join
# no.1-2
# 商品購入データのitem列を結合キーにして、imasterのindexを突き合わせて商品(item)列を結合する。
# join()は、このように結合される側(leftと呼ぶ)の結合キーは列で、結合する側(rightと呼ぶ)がindexであるような結合となる場合が多い。
# また、２つめの実行例のように、join()で実現できることはmergeでも実現可能であるが、パラメータの指定方法が少し複雑になる。
# mergeで指定したパラメータについては後述している。
>>> print(tra.join(imaster, on='item'))
>>> print(tra.merge(imaster, left_on='item', right_index=True, how='left'))''')
print(tra.join(imaster, on='item'))
print(tra.merge(imaster, left_on='item', right_index=True, how='left'))

print('''
## concat
# no.1-3
# 顧客マスターを縦に連結する例。
# このように同じ列で構成される複数のDataFrameを縦に連結するケースが多いが、より複雑な連結も実現できる。詳細は後述する。
# 連結された行indexは元のindexがそのまま採用されるため、indexが重複していることに注意する。
>>> print(pd.concat([cmaster, cmaster]))''')
print(pd.concat([cmaster, cmaster]))



# 入力データ
# traは顧客がどの商品を購入したかに関するトランザクションデータである。
# custは顧客の年齢と性別を示した顧客マスターデータである。
# 一方でitemは商品の値段を示した商品マスターでcustomer列は行indexとしてセットしている。
>>> tra = pd.DataFrame(
        {'customer':['A', 'A', 'A', 'B', 'C', 'C', 'C'],
         'item':['coffee', 'bread', 'milk', 'juice', 'tea', 'milk', 'bread']}
    )
>>> print(tra)
  customer    item
0        A  coffee
1        A   bread
2        A    milk
3        B   juice
4        C     tea
5        C    milk
6        C   bread

>>> cmaster = pd.DataFrame(
        {'customer':['A', 'C'],
         'age':[40, 25],
         'gender':['female', 'male']}
    )
>>> print(cmaster)
  customer  age  gender
0        A   40  female
1        C   25    male

>>> imaster = pd.DataFrame(
        {'item':['coffee', 'bread', 'milk', 'tea'],
         'price':[100, 120, 110, 80]}
     )
>>> imaster = imaster.set_index('item')
>>> print(imaster)
        price
item         
coffee    100
bread     120
milk      110
tea        80

## merge
# no.1-1
# 商品購入データのcustomer列と顧客マス

---
# 2. 列の結合(merge, join)
mergeとjoinの違いは1節で見たように、結合キーの扱いによるところが多いが、2つのメソッドの利用について押さえておくべきポイントは、1) 結合キーの型(indexか列)、2) 結合キーの重複があった場合のどのように動作するか、3) 結合キーのleftとrightが一致しない場合にどのように動作するか、の3点である。
1)の結合キーの型については、leftとrightで、indexと列の組み合わせで4通りある。
2)の結合キーの重複については、leftとrightで計4つの組み合わせがある。
leftもrightも重複がない結合は1:1結合と呼ばれ、leftにのみ重複がある結合をm:1、rightのみに重複がある結合は1:m、そして両者に重複がある結合はm:m結合と呼ばれる(mはmanyの頭文字)。
そして、3)のleftとrightの結合キーの不一致については、left/rightどちらの不一致の行を残すかによって、inner, left, right, outerの4つの組み合わせがある。
innerは両者の不一致キーは残さなず(すなわち共通の結合キーのみを残す)、leftはleftの不一致のみ残し、rightはrightの不一致のみ残し、outerは両者の不一致を残す。
結合キーの列の型については2-1で、結合キーの重複については2-2で、結合キーの不一致については2-3で扱う。
またleftとrightに指定できるデータ型は下表に示すとおりである。

<div style="width:1000px; border:1px solid #cc0000;">
  <table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:25%;">
      <col style="width:15%;">
      <col style="width:15%;">
      <col style="width:45%;">
    </colgroup>
      
  <tbody>
    <tr><th align="left">メソッド</th><th align="left">leftに指定可能な型</th><th align="left">rightに指定可能な型</th><th align="left">備考</th></tr>
    <tr><td><TT>left.merge(right)</TT><BR>
            <TT> pd.merge(left,right)</TT></td>
        <td>DataFrame</td>
        <td>DataFrame<BR>Series</td>
        <td>rightにSeriesを指定する時はname=を設定しなければならない(named Series)。Seriesはそのインデックスをキーとして結合される。</td>
    </tr>
      
    <tr><td><TT> left.join(right)</TT></td>
        <td>DataFrame</td>
        <td>DataFrame<BR>Series<BR>DataFrameのリスト</td>
        <td>rightにSeriesを指定する時はname=を設定しなければならない。</td>
    </tr>
</tbody>
</table>
    
</div>


---
# 2-1. 結合キーの型(index vs 列)

### 入力データ
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
l = pd.DataFrame({'k':['A', 'B', 'C'], 'lv':[1, 2, 3]})
r = pd.DataFrame({'k':['A', 'B', 'C'], 'rv':[10, 20, 30]})
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:5%;">
      <col style="width:40%;">
      <col style="width:35%;">
      <col style="width:20%;">
    </colgroup>
<tbody>

<tr><th align="left">no.</th><th align="left">実行コード</th><th align="center">入力 / 出力</th><th>備考</th></tr>

<tr><td colspan=4 bgcolor="#8fbc8f"><b>column to columnの結合</b></td></tr>   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-1-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(l.merge(r, on='k'))
</pre></td>
<td align="center"><img width="60%" src="./figures/join/join2-1-1.png"></td>
<td>leftの結合キーが列(column)、rightの結合キーも列の例(column)。
rightが列の場合はjoin()は利用できない。
</td>
</tr>
    
<tr><td colspan=4 bgcolor="#8fbc8f"><b>column to indexの結合</b></td></tr>    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-1-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
ri = r.set_index('k')
print(l.merge(ri, left_on='k', right_index=True))
print(l.join(ri, on='k'))
</pre></td>
<td align="center"><img width="60%" src="./figures/join/join2-1-2.png"></td>
<td>
leftの結合キーが列(column)、rightの結合キーがindexの例。
mergeについて、riはindexに軸名('k')がついているので、l.merge(ri, on='k') でも可能。
</td>
</tr>
  
<tr><td colspan=4 bgcolor="#8fbc8f"><b>index to columnの結合</b></td></tr>    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-1-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
li = l.set_index('k')
print(li.merge(r, left_index=True, right_on='k'))
</pre></td>
<td align="center"><img width="60%" src="./figures/join/join2-1-3.png"></td>
<td>
leftの結合キーがindex、rightの結合キーが列(column)の例。
rightが列の場合はjoin()は利用できない。mergeについて、liはindexに軸名('k')がついているので、li.merge(r, on='k') でも可能。
leftの結合キーがindexだが、結合結果のindexはrightのindexとなる。
</td>
</tr>
    
<tr><td colspan=4 bgcolor="#8fbc8f"><b>index to indexの結合</b></td></tr>    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-1-４</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
li = l.set_index('k')
ri = r.set_index('k')
print(li.merge(ri, left_index=True, right_index=True))
print(li.join(ri))
</pre></td>
<td align="center"><img width="45%" src="./figures/join/join2-1-4.png"></td>
<td>
left/rightの両方とも結合キーがindexの例。
mergeについて、li/riともにindexに名前がついているので、li.merge(ri, on='k') でも可能。
left/right両方とも結合キーがindexの場合、join()のon=は省略できる。
</td>
</tr>
 
</tbody>
</table>


In [3]:
print('''
#　leftとrightの型について結合キーの1:1(one to one)結合
# 入力データ
>>> l = pd.DataFrame({'k':['A', 'B', 'C'], 'lv':[1, 2, 3]})
>>> print(l)
''')
l = pd.DataFrame({'k':['A', 'B', 'C'], 'lv':[1, 2, 3]})
print(l)

print('''
>>> r = pd.DataFrame({'k':['A', 'B', 'C'], 'rv':[10, 20, 30]})
>>> print(r)
''')
r = pd.DataFrame({'k':['A', 'B', 'C'], 'rv':[10, 20, 30]})
print(r)



print('''
# no.2-1-1
# column to columnの結合。
# leftの結合キーが列(column)、rightの結合キーも列の例(column)。
# rightが列の場合はjoin()は利用できない。
>>> print(l.merge(r, on='k'))''')
print(l.merge(r, on='k'))

print('''
# no.2-1-2
# column to indexの結合。
# leftの結合キーが列(column)、rightの結合キーがindexの例。
# mergeについて、riはindexに名前がついているので、l.merge(ri, on='k') でも可能。
>>> ri = r.set_index('k')
>>> print(l.merge(ri, left_on='k', right_index=True))
>>> print(l.join(ri, on='k'))''')
ri = r.set_index('k')
print(l.merge(ri, left_on='k', right_index=True))
print(l.join(ri, on='k'))

print('''
# no.2-1-3
# index to columnの結合。
# leftの結合キーがindex、rightの結合キーが列(column)の例。
# rightが列の場合はjoin()は利用できない。
# mergeについて、riはindexに名前がついているので、li.merge(r, on='k') でも可能。
# leftの結合キーがindexだが、結合結果のindexはrightのindexとなる。
>>> li = l.set_index('k')
>>> print(li.merge(r, left_index=True, right_on='k'))''')
li = l.set_index('k')
print(li.merge(r, left_index=True, right_on='k'))

print('''
# no.2-1-4
# index to indexの結合。
# left/rightの両方とも結合キーがindexの例。
# mergeについて、li/riともにindexに名前がついているので、li.merge(ri, on='k') でも可能。
# left/right両方とも結合キーがindexの場合、join()のon=は省略できる。
>>> li = l.set_index('k')
>>> ri = r.set_index('k')
>>> print(li.merge(ri, left_index=True, right_index=True))
>>> print(li.join(ri))''')
li = l.set_index('k')
ri = r.set_index('k')
print(li.merge(ri, left_index=True, right_index=True))
print(li.join(ri))




#　leftとrightの型について結合キーの1:1(one to one)結合
# 入力データ
>>> l = pd.DataFrame({'k':['A', 'B', 'C'], 'lv':[1, 2, 3]})
>>> print(l)

   k  lv
0  A   1
1  B   2
2  C   3

>>> r = pd.DataFrame({'k':['A', 'B', 'C'], 'rv':[10, 20, 30]})
>>> print(r)

   k  rv
0  A  10
1  B  20
2  C  30

# no.2-1-1
# column to columnの結合。
# leftの結合キーが列(column)、rightの結合キーも列の例(column)。
# rightが列の場合はjoin()は利用できない。
>>> print(l.merge(r, on='k'))
   k  lv  rv
0  A   1  10
1  B   2  20
2  C   3  30

# no.2-1-2
# column to indexの結合。
# leftの結合キーが列(column)、rightの結合キーがindexの例。
# mergeについて、riはindexに名前がついているので、l.merge(ri, on='k') でも可能。
>>> ri = r.set_index('k')
>>> print(l.merge(ri, left_on='k', right_index=True))
>>> print(l.join(ri, on='k'))
   k  lv  rv
0  A   1  10
1  B   2  20
2  C   3  30
   k  lv  rv
0  A   1  10
1  B   2  20
2  C   3  30

# no.2-1-3
# index to columnの結合。
# leftの結合キーがindex、rightの結合キーが列(column)の例。
# rightが列の場合はjoin()は利用できない。
# mergeについて、riはindexに名前がついているので、li.merge(r, on='k') でも可能。
# leftの結合キーがindexだが、結合結果の

---
# 2-2. 結合キーの重複(1:1, m:1, m:m)

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
l = pd.DataFrame({'k':['A', 'B', 'C'], 'lv':[1, 2, 3]})
r = pd.DataFrame({'k':['A', 'B', 'C'], 'rv':[10, 20, 30]})
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
   
    <colgroup>
      <col style="width:5%;">
      <col style="width:40%;">
      <col style="width:35%;">  
      <col style="width:20%;">
    </colgroup>
    
<tbody>

<tr><th align="left">no</th><th align="left">実行コード</th><th align="center">入力 / 出力</th><th align="center">備考</th></tr>
    
<tr><td colspan=6 bgcolor="#8fbc8f"><TT><b>1 : 1(one to one)結合</b></TT></td></tr>  
  
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.merge(l, r, on='k'))
</pre></td>
<td align="center"><img width="60%" src="./figures/join/join2-2-1.png"></td>
<td>mergeによる1:1結合。'k'を結合キーとしてlとrを結合。</td></tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(l.merge(r,on='k'))
</pre></td>
<td align="center"><img width="60%" src="./figures/join/join2-2-2.png"></td>
<td>2-2-1と同じ動きをする別の書き方</td></tr>    

<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
li = l.set_index('k')
ri = r.set_index('k')
print(li.join(ri))
</pre></td>
<td align="center"><img width="40%" src="./figures/join/join2-2-3.png"></td>
<td>joinによる結合。'k'をindexに指定してlとrを結合。</td></tr>
 
</tbody>
</table>

 
### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
x = pd.DataFrame({'k':['A', 'B', 'C'], 'xv':[100, 200, 300]})
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
   
    <colgroup>
      <col style="width:5%;">
      <col style="width:40%;">
      <col style="width:35%;">  
      <col style="width:20%;">
    </colgroup>
    
<tbody>
        
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2-4</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
li = l.set_index('k')
ri = r.set_index('k')
xi = x.set_index('k')
print(li.join([ri, x]))
</pre></td>
<td align="center"><img width="60%" src="./figures/join/join2-2-4.png"></td>
<td>
join()による複数のDataFrameの結合。'k'をindexに指定してlとrとxを結合。
pd.join([li, ri, xi)のようには書けない。
</td></tr>
    
</tbody>
</table>
 

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
l = pd.DataFrame({'k':['A', 'A', 'B'], 'lv':[1, 2, 3]})
r = pd.DataFrame({'k':['A', 'B'], 'rv':[10, 20]})
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
   
    <colgroup>
      <col style="width:5%;">
      <col style="width:40%;">
      <col style="width:35%;">  
      <col style="width:20%;">
    </colgroup>
    
<tbody>
    
<tr><td colspan=6 bgcolor="#8fbc8f"><TT><b>n : 1(many to one)結合</b></TT></td></tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2-5</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(l.merge(r, on='k'))
</pre></td>
<td align="center"><img width="50%" src="./figures/join/join2-2-5.png"></td>
<td>mergeの結合。lの結合キーのデータ(A)が重複している。
このような場合、重複した全ての行にleftの同じキーの行が結合される。</td></tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2-6</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
li = l.set_index('k')
ri = r.set_index('k')
print(li.join(ri))
</pre></td>
<td align="center"><img width="40%" src="./figures/join/join2-2-6.png"></td>
<td>joinも考え方は同じ。</td></tr>
    
</tbody>
</table>


### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
l = pd.DataFrame({'k':['A', 'A', 'B'], 'lv':[1, 2, 3]})
r = pd.DataFrame({'k':['A', 'A', 'B'], 'rv':[10, 20, 30]})
</pre>    

<table border="1" style="table-layout:fixed;width:100%;">
   
    <colgroup>
      <col style="width:5%;">
      <col style="width:40%;">
      <col style="width:35%;">  
      <col style="width:20%;">
    </colgroup>
    
<tbody>    
<tr><td colspan=6 bgcolor="#8fbc8f"><TT><b>m:m(many to many)結合</b></TT></td></tr>    

<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2-7</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(l.merge(r, on='k'))
</pre></td>
<td align="center"><img width="60%" src="./figures/join/join2-2-7.png"></td>
<td>l,rともに結合キーにAが2つある。このような場合、全ての組み合わせ(2×2=4通り)の全ての結合(全結合)となる。</td></tr>    
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2-8</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
li = l.set_index('k')
ri = r.set_index('k')
print(li.join(ri))
</pre></td>
<td align="center"><img width="40%" src="./figures/join/join2-2-8.png"></td>
<td>joinも考え方は同じ。</td></tr>    
 
</tbody>
</table>


In [4]:

print('''
#　1:1(one to one)結合
# 入力データ
>>> l = pd.DataFrame({'k':['A', 'B', 'C'], 'lv':[1, 2, 3]})
>>> r = pd.DataFrame({'k':['A', 'B', 'C'], 'rv':[10, 20, 30]})
>>> print(l)
>>> print(r)''')
l = pd.DataFrame({'k':['A', 'B', 'C'], 'lv':[1, 2, 3]})
r = pd.DataFrame({'k':['A', 'B', 'C'], 'rv':[10, 20, 30]})
print(l)
print(r)

print('''
# no.2-2-1 
# mergeの1:1結合。l,rともに結合キーに重複はなく、結合キーkの値A,B,Cで1:1で結合する。
>>> print(pd.merge(l, r, on='k'))''')
print(pd.merge(l, r, on='k'))

print('''
# no.2-2-2
# no.2-2-1と同じ動きをする別の書き方。
>>> print(l.merge(r,on='k'))''')
print(l.merge(r,on='k'))

print('''
# no.2-2-3 
# join()による'k'をindexにした同士のDataFrameの結合。
>>> li = l.set_index('k')
>>> ri = r.set_index('k')
>>> print(li.join(ri))''')
li = l.set_index('k')
ri = r.set_index('k')
print(li.join(ri))

print('''
# no.2-2-4
# join()による複数のDataFrameの結合。'k'をindexに指定してlとrとxを結合。 
# pd.join([li, ri, xi)のようには書けない。

# 入力データ
>>> x = pd.DataFrame({'k':['A', 'B', 'C'], 'xv':[100, 200, 300]})
>>> print(x)''')
x = pd.DataFrame({'k':['A', 'B', 'C'], 'xv':[100, 200, 300]})
print(x)
print('''
>>> li = l.set_index('k')
>>> ri = r.set_index('k')
>>> xi = x.set_index('k')
>>> print(li.join([ri, xi]))''')
li = l.set_index('k')
ri = r.set_index('k')
xi = x.set_index('k')
print(li.join([ri, xi]))

print('''
#　m:1(many to one)結合
# 入力データ
>>> l = pd.DataFrame({'k':['A', 'A', 'B'], 'lv':[1, 2, 3]})
>>> r = pd.DataFrame({'k':['A', 'B'], 'rv':[10, 20]})
>>> print(l)
>>> print(r)''')
l = pd.DataFrame({'k':['A', 'A', 'B'], 'lv':[1, 2, 3]})
r = pd.DataFrame({'k':['A', 'B'], 'rv':[10, 20]})
print(l)
print(r)


print('''
# no.2-2-5
# leftの結合キーはAが2行に存在する(重複がある)。このような場合、それぞれのAについてrightのAを結合する。
>>> l.merge(r, on='k')''')
print(l.merge(r, on='k'))

print('''
# no.2-2-6
# joinも考え方は同じ。
>>> li = l.set_index('k')
>>> ri = r.set_index('k')
>>> print(li.join(ri))''')
li = l.set_index('k')
ri = r.set_index('k')
print(li.join(ri))

print('''
# m:m(many to many)結合
# 入力データ
>>> l = pd.DataFrame({'k':['A', 'A', 'B'], 'lv':[1, 2, 3]})
>>> r = pd.DataFrame({'k':['A', 'A', 'B'], 'rv':[10, 20, 30]})
print(l)
print(r)''')
l = pd.DataFrame({'k':['A', 'A', 'B'], 'lv':[1, 2, 3]})
r = pd.DataFrame({'k':['A', 'A', 'B'], 'rv':[10, 20, 30]})
print(l)
print(r)


print('''
# no.2-2-7
# l,rともに結合キーにAが2つある。このような場合、全ての組み合わせ(2×2=4通り)の全ての結合(全結合)となる。
>>> print(l.merge(r, on='k'))''')
print(l.merge(r, on='k'))

print('''
# no.2-2-8
# joinも考え方は同じ。
>>> print(li.join(ri))''')
li = l.set_index('k')
ri = r.set_index('k')
print(li.join(ri))




#　1:1(one to one)結合
# 入力データ
>>> l = pd.DataFrame({'k':['A', 'B', 'C'], 'lv':[1, 2, 3]})
>>> r = pd.DataFrame({'k':['A', 'B', 'C'], 'rv':[10, 20, 30]})
>>> print(l)
>>> print(r)
   k  lv
0  A   1
1  B   2
2  C   3
   k  rv
0  A  10
1  B  20
2  C  30

# no.2-2-1 
# mergeの1:1結合。l,rともに結合キーに重複はなく、結合キーkの値A,B,Cで1:1で結合する。
>>> print(pd.merge(l, r, on='k'))
   k  lv  rv
0  A   1  10
1  B   2  20
2  C   3  30

# no.2-2-2
# no.2-2-1と同じ動きをする別の書き方。
>>> print(l.merge(r,on='k'))
   k  lv  rv
0  A   1  10
1  B   2  20
2  C   3  30

# no.2-2-3 
# join()による'k'をindexにした同士のDataFrameの結合。
>>> li = l.set_index('k')
>>> ri = r.set_index('k')
>>> print(li.join(ri))
   lv  rv
k        
A   1  10
B   2  20
C   3  30

# no.2-2-4
# join()による複数のDataFrameの結合。'k'をindexに指定してlとrとxを結合。 
# pd.join([li, ri, xi)のようには書けない。

# 入力データ
>>> x = pd.DataFrame({'k':['A', 'B', 'C'], 'xv':[100, 200, 300]})
>>> print(x)
   k   xv
0  A  100
1  B  200
2  C  300

>>> li = l.set_index('k')
>>> ri = r.set_index('k')
>>> xi = x.set_index('k

---
## 2-3. 結合キーの不一致(inner, left, right, outer)

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dfL = pd.DataFrame({'k':['A', 'B', 'C', 'D'], 'lv':[1, 2, 3, 4]})
dfR = pd.DataFrame({'k':['B', 'D', 'D', 'E'], 'rv':[10, 20, 30, 40]})
dfLi = dfL.set_index('k')
dfRi = dfR.set_index('k')
</pre>


<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:5%;">
      <col style="width:29%;">
      <col style="width:23%;">  
      <col style="width:23%;"> 
      <col style="width:20%;">
    </colgroup>
    
<tbody>

<tr><th align="left">no</th><th align="left">実行コード</th><th align="left">入力 / 出力</th><th align="left">入力 / 出力</th><th align="left">備考</th></tr>
  
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-3-1</td>
<td>INNER JOIN（内部結合）※mergeのデフォルト
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dfL.merge(dfR, on='k'))
print(dfLi.join(dfRi, how='inner'))
</pre></td>
<td><img width="100%" src="./figures/join/join2-3-1.png"></td>
<td><img width="100%" src="./figures/join/join2-3-1j.png"></td>
<td>'k'列を結合キーとして、両方のDataFrameで一致する行のみを抽出して結合させる。merge()はinnerがデフォルトのため指定する必要はないが、join()はLEFT OUTER JOINがデフォルトのため、how='inner'を指定する必要がある。</td></tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-3-2</td>
<td>LEFT OUTER JOIN(左外部結合)
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dfL.merge(dfR, on='k', how='left'))
print(dfLi.join(dfRi))
</pre></td>
<td><img width="90%" src="./figures/join/join2-3-2.png"></td>
<td><img width="100%" src="./figures/join/join2-3-2j.png"></td>
<td>how='left'を指定すれば、leftの結合キーは全て出力されるが、rightにのみ存在する結合キーは出力されない。leftにあってrightにない行にはNAが出力される。join()はhow='left'がデフォルトのため省略できる。</td></tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-3-3</td>
<td>RIGHT OUTER JOIN(右外部結合)
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dfL.merge(dfR, on='k', how='right'))
print(dfLi.join(dfRi, how='right'))
</pre></td>
<td><img width="90%" src="./figures/join/join2-3-3.png"></td>
<td><img width="100%" src="./figures/join/join2-3-3j.png"></td>
<td>how='right'を指定すれば、how='left'とは逆に、rightの結合キーは全て出力されるが、leftにのみ存在する結合キーは出力されない。rightにあってleftにない行にはNAが出力される。</td></tr>    

<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-3-4</td>
<td>FULL OUTER JOIN(完全外部結合)
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dfL.merge(dfR, on='k', how='outer'))
print(dfLi.join(dfRi, how='outer'))
</pre></td>
<td><img width="100%" src="./figures/join/join2-3-4.png"></td>
<td><img width="90%" src="./figures/join/join2-3-4j.png"></td>
<td>how='outer'を指定すれば、left/right両方の結合キーが全て出力される。お互いに存在しない行にはNAが出力される。rightにあってleftにない行にはNAが出力される。</td></tr>    
 
</tbody>
</table>

In [5]:
print('''
# 入力データ
>>> dfL = pd.DataFrame({'k':['A', 'B', 'C', 'D'], 'lv':[1, 2, 3, 4]})
>>> dfR = pd.DataFrame({'k':['B', 'D', 'D', 'E'], 'rv':[10, 20, 30, 40]})
>>> dfLi = dfL.set_index('k')
>>> dfRi = dfR.set_index('k')
>>> print(dfL)
>>> print(dfR)
>>> print(dfLi)
>>> print(dfRi)''')
dfL = pd.DataFrame({'k':['A', 'B', 'C', 'D'], 'lv':[1, 2, 3, 4]})
dfR = pd.DataFrame({'k':['B', 'D', 'D', 'E'], 'rv':[10, 20, 30, 40]})
dfLi = dfL.set_index('k')
dfRi = dfR.set_index('k')
print(dfL)
print(dfR)
print(dfLi)
print(dfRi)

print('''
# no.2-3-1 
# INNER JOIN
# how='inner'を指定すれば、結合キーがleft/rightに共通の行のみが出力される。
# merge()はinnerがデフォルトのため指定する必要はないが、join()はLEFT OUTER JOINがデフォルトのため、how='inner'を指定する必要がある。
print(dfL.merge(dfR, on='k'))
print(dfLi.join(dfRi, how='inner')''')
print(dfL.merge(dfR, on='k'))
print(dfLi.join(dfRi, how='inner'))

print('''
# no.2-3-2
# LEFT OUTER JOIN
# how='left'を指定すれば、leftの結合キーは全て出力されるが、rightにのみ存在する結合キーは出力されない。
# leftにあってrightにない行にはNAが出力される。
# join()はhow='left'がデフォルトのため省略できる。
print(dfL.merge(dfR, on='k', how='left'))
print(dfLi.join(dfRi)''')
print(dfL.merge(dfR, on='k', how='left'))
print(dfLi.join(dfRi))

print('''
# no.2-3-3
# RIGHT OUTER JOIN
# how='right'を指定すれば、how='left'とは逆に、rightの結合キーは全て出力されるが、leftにのみ存在する結合キーは出力されない。
# rightにあってleftにない行にはNAが出力される。
print(dfL.merge(dfR, on='k', how='right'))
print(dfLi.join(dfRi, how='right')''')
print(dfL.merge(dfR, on='k', how='right'))
print(dfLi.join(dfRi, how='right'))

print('''
# no.2-3-4 
# FULL OUTER JOIN
# how='outer'を指定すれば、left/right両方の結合キーが全て出力される。お互いに存在しない行にはNAが出力される。
# rightにあってleftにない行にはNAが出力される。
>>> print(dfL.merge(dfR, on='k', how='outer'))
>>> print(dfLi.join(dfRi, how='outer'))''')
print(dfL.merge(dfR, on='k', how='outer'))
print(dfLi.join(dfRi, how='outer'))
      


# 入力データ
>>> dfL = pd.DataFrame({'k':['A', 'B', 'C', 'D'], 'lv':[1, 2, 3, 4]})
>>> dfR = pd.DataFrame({'k':['B', 'D', 'D', 'E'], 'rv':[10, 20, 30, 40]})
>>> dfLi = dfL.set_index('k')
>>> dfRi = dfR.set_index('k')
>>> print(dfL)
>>> print(dfR)
>>> print(dfLi)
>>> print(dfRi)
   k  lv
0  A   1
1  B   2
2  C   3
3  D   4
   k  rv
0  B  10
1  D  20
2  D  30
3  E  40
   lv
k    
A   1
B   2
C   3
D   4
   rv
k    
B  10
D  20
D  30
E  40

# no.2-3-1 
# INNER JOIN
# how='inner'を指定すれば、結合キーがleft/rightに共通の行のみが出力される。
# merge()はinnerがデフォルトのため指定する必要はないが、join()はLEFT OUTER JOINがデフォルトのため、how='inner'を指定する必要がある。
print(dfL.merge(dfR, on='k'))
print(dfLi.join(dfRi, how='inner')
   k  lv  rv
0  B   2  10
1  D   4  20
2  D   4  30
   lv  rv
k        
B   2  10
D   4  20
D   4  30

# no.2-3-2
# LEFT OUTER JOIN
# how='left'を指定すれば、leftの結合キーは全て出力されるが、rightにのみ存在する結合キーは出力されない。
# leftにあってrightにない行にはNAが出力される。
# join()はhow='left'がデフォルトのため省略できる。
print(dfL.merge(dfR, on='k', how='left'))
print(dfLi.join(dfRi)
   k  l

---
### 2-4. Seriesの結合

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dfL = pd.DataFrame({'k':['A', 'B', 'C', 'D'], 'lv':[1, 2, 3, 4]})
dfLi = dfL.set_index('k')
sr = pd.Series([10, 20, 30, 40], index=['B', 'D', 'D', 'E'], name='sv')
</pre>

 
<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:5%;">
      <col style="width:35%;">
      <col style="width:25%;">  
      <col style="width:35%;">
    </colgroup>
    
<tbody>                              
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-4-1</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dfL.merge(sr, left_on='k', right_index=True))
</pre></td>
<td><img width="80%" src="./figures/join/join2-4-1.png"></td>
<td>merge()でSeriesを結合する。重複の結合キーと結合キーの不一致については、DataFrameの結合と同様である。</td></tr>
   
<tbody>                              
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-4-2</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dfL.join(sr, on='k'))
</pre></td>
<td><img width="80%" src="./figures/join/join2-4-2.png"></td>
<td>join()はhow='left'がデフォルトのため省略できる。重複の結合キーと結合キーの不一致についても、DataFrameの結合と同様である。</td></tr>
    
    
        
<tbody>                              
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-4-3</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dfLi.join(sr))
</pre></td>
<td><img width="80%" src="./figures/join/join2-4-3.png"></td>
<td>DataFrameと同様に、index to indexのjoin()であればon=を省略可能。</td></tr>
   
</tbody>
</table>

In [6]:
### Seriesの結合
print('''
# 入力データ
DataFrameにSeriesを結合する場合、
Seriesに結合キーをindexとして設定し、かつnameをつけなければならない。そのnameが結合後のDataFrame上の列名となる
>>> dfL = pd.DataFrame({'k':['A', 'B', 'C', 'D'], 'lv':[1, 2, 3, 4]})
>>> dfLi = dfL.set_index('k')
>>> sr = pd.Series([10, 20, 30, 40], index=['B', 'D', 'D', 'E'], name='sv')
>>> print(dfL)
>>> print(dfLi)
>>> print(sr)''')
dfL = pd.DataFrame({'k':['A', 'B', 'C', 'D'], 'lv':[1, 2, 3, 4]})
dfLi = dfL.set_index('k')
sr = pd.Series([10, 20, 30, 40], index=['B', 'D', 'D', 'E'], name='sv')
print(dfL)
print(dfLi)
print(sr)


print('''
# no.2-4-1
# merge()でSeriesを結合する。重複の結合キーと結合キーの不一致については、DataFrameの結合と同様である。
>>> print(dfL.merge(sr, left_on='k', right_index=True))''')
print(dfL.merge(sr, left_on='k', right_index=True))

print('''
# no.2-4-2
# join()はhow='left'がデフォルトのため省略できる。重複の結合キーと結合キーの不一致についても、DataFrameの結合と同様である。
print(dfL.join(sr, on='k'))''')
print(dfL.join(sr, on='k'))

print('''
# no.2-4-3
# DataFrameと同様に、index to indexのjoin()であればon=を省略可能。
>>> print(dfLi.join(sr))''')
print(dfLi.join(sr))



# 入力データ
DataFrameにSeriesを結合する場合、
Seriesに結合キーをindexとして設定し、かつnameをつけなければならない。そのnameが結合後のDataFrame上の列名となる
>>> dfL = pd.DataFrame({'k':['A', 'B', 'C', 'D'], 'lv':[1, 2, 3, 4]})
>>> dfLi = dfL.set_index('k')
>>> sr = pd.Series([10, 20, 30, 40], index=['B', 'D', 'D', 'E'], name='sv')
>>> print(dfL)
>>> print(dfLi)
>>> print(sr)
   k  lv
0  A   1
1  B   2
2  C   3
3  D   4
   lv
k    
A   1
B   2
C   3
D   4
B    10
D    20
D    30
E    40
Name: sv, dtype: int64

# no.2-4-1
# merge()でSeriesを結合する。重複の結合キーと結合キーの不一致については、DataFrameの結合と同様である。
>>> print(dfL.merge(sr, left_on='k', right_index=True))
   k  lv  sv
1  B   2  10
3  D   4  20
3  D   4  30

# no.2-4-2
# join()はhow='left'がデフォルトのため省略できる。重複の結合キーと結合キーの不一致についても、DataFrameの結合と同様である。
print(dfL.join(sr, on='k'))
   k  lv    sv
0  A   1   NaN
1  B   2  10.0
2  C   3   NaN
3  D   4  20.0
3  D   4  30.0

# no.2-4-3
# DataFrameと同様に、index to indexのjoin()であればon=を省略可能。
>>> print(dfLi.join(sr))
   lv    sv
A   1   NaN
B   2  10.0
C   3   NaN
D   4  20.0
D

---
# 3.行と列の連結(concat)
`merge()`や`join()`は、結合キーを突き合わせながら２つのデータを列として結合するメソッドであった。
`concat()`も似たような動きをするが、基本的なイメージは、2つのデータを行or列方向にラベルを結合キーにして結合することである。
以下の左図の左列は２つのDataFrameを行方向に連結する典型例で、右図の上行は列方向に連結する典型例である。
注意が必要なことは、行方向で連結する場合を例にとると、結合する列のラベルが異なる時の動作である。
行方向には単純にそのまま連結するが、列ラベルが異なる場合、ラベルが同一なもの同士はそのラベル名で連結するが、
異なるラベルについては、2つの方法があり、1つは、同一ラベルの列のみを出力する方法で(innerによる連結)、
他方は、異なるラベルは別の列として出力する方法である(outerによる連結)。
これは列方向の連結でも同様に動作する。すこし混乱するのは、列の連結はjoin()でも同様の結果を得ることができる点である。
<div style="width:1000px">
  <table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:50%;">
      <col style="width:50%;">
    </colgroup>
<tbody>
<tr><th align="center">縦連結（行方向）の連結</th><th align="center">横連結（列方向）の連結</th></tr>
    <tr>
        <td><img width="100%" src="./figures/join/join2_concat_row.png"></td>
        <td><img width="90%" src="./figures/join/join2_concat_col.png"></td>
    </tr>
</tbody>
</table>
    
</div>

### 縦連結の入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dt0 = pd.DataFrame({'a':[1, 2], 'b':[1, 2], 'c':[1, 2]})
dt1 = pd.DataFrame({'a':[3, 4], 'b':[3, 4], 'c':[3, 4]})
dt2 = pd.DataFrame({'b':[3, 4], 'c':[3, 4], 'd':[3, 4]})
</pre>


<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:5%;">
      <col style="width:35%;">
      <col style="width:40%;">  
      <col style="width:20%;">
    </colgroup>
    
<tbody>

<tr><th align="left">no</th><th align="left">実行コード</th><th align="left">入力 / 出力</th><th  align="left">備考</th></tr>   
<tr><td colspan=4 bgcolor="#8fbc8f"><TT><b>縦連結 (concat axis=0)</b></TT></td></tr>  
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-1</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([dt0, dt1]))
</pre></td>
<td><img width="80%" src="./figures/join/join3-1.png"></td>
<td>axis=を省略すると縦連結</td></tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-2</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dt0.append(dt1))
</pre></td>
<td><img width="80%" src="./figures/join/join3-2.png"></td>
<td>一つずつ連結するならappendを使っても同じ</td></tr>    
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-3</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([dt0, dt1], ignore_index=True))
</pre></td>
<td><img width="85%" src="./figures/join/join3-3.png"></td>
<td>ignore_index=Trueとすればindexはリセットされる。</td></tr>    
        
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-4</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([dt0, dt2]))
</pre></td>
<td><img width="100%" src="./figures/join/join3-4.png"></td>
<td>列名が一部異なるDataFrameの連結。join='outer'がデフォルトで、お互いに存在しない列にはNAが出力される。</td></tr>    

<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-5</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(dt0.append(dt2))
</pre></td>
<td><img width="100%" src="./figures/join/join3-5.png"></td>
<td>appendはouterのみで動作する。</td></tr>  
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-6</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([dt0, dt2], join='inner'))
</pre></td>
<td><img width="90%" src="./figures/join/join3-6.png"></td>
<td>join='inner'を指定した場合、共通の列名のみ連結される。</td></tr>  
</tbody>
</table>

### 横連結の入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dc0 = pd.DataFrame({'a':['a', 'a', 'a'], 'b':['b', 'b', 'b']}, index=[1, 2, 3])
dc1 = pd.DataFrame({'c':['c', 'c', 'c'], 'd':['d', 'd', 'd']}, index=[1, 2, 3])
dc2 = pd.DataFrame({'c':['c', 'c', 'c'], 'd':['d', 'd', 'd']}, index=[2, 3, 4])
</pre>


<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:5%;">
      <col style="width:35%;">
      <col style="width:25%;">  
      <col style="width:35%;">
    </colgroup>
    
<tbody>
    
<tr><th align="left">no</th><th align="left">実行コード</th><th align="left">入力 / 出力</th><th  align="left">備考</th></tr>   
<tr><td colspan=4 bgcolor="#8fbc8f"><TT><b>横連結(concat axis=1)</b></TT></td></tr>  
   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-7</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([dc0, dc1], axis=1))
</pre></td>
<td><img width="100%" src="./figures/join/join3-7.png"></td>
<td>axis=1で横連結</td></tr>  
   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-8</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([dc0, dc2], axis=1))
</pre></td>
<td><img width="100%" src="./figures/join/join3-8.png"></td>
<td>行ラベルが一部異なるDataFrameの連結。join='outer'がデフォルトで、お互いに存在しない行にはNAが出力される。</td></tr>  
   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-9</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([dc0, dc2], axis=1, join='inner'))
</pre></td>
<td><img width="100%" src="./figures/join/join3-9.png"></td>
<td>join='inner'を指定した場合、共通の行ラベルのみ連結される。</td></tr> 
    
</tbody>
</table>

### 横連結の入力データ(seriesの連結)

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
s1 = pd.Series([1, 2, 3, 4], index=[0, 1, 2, 3], name='s1')
s2 = pd.Series([1, 2, 3, 4], index=[3, 1, 0, 4], name='s2')
</pre>


<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:5%;">
      <col style="width:35%;">
      <col style="width:25%;">  
      <col style="width:35%;">
    </colgroup>
    
<tbody>
    
<tr><th align="left">no</th><th align="left">実行コード</th><th align="left">入力 / 出力</th><th  align="left">備考</th></tr> 
<tr><td colspan=6 bgcolor="#8fbc8f"><TT><b>横連結(concat axis=1) Seriesの連結</b></TT></td></tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-10</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([s1, s2]))
</pre></td>
<td><img width="100%" src="./figures/join/join3-10.png"></td>
<td>
Seriesの縦連結。
nameが異なるとNoneにセットされ、同じであれば受け継がれる。
結果もSeriesとなる。
</td></tr>        
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-11</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([s1, s2], axis=1))
</pre></td>
<td><img width="90%" src="./figures/join/join3-11.png"></td>
<td>axis=1で横連結すると結果はDataFrameになる。</td></tr>     
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-12</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat({'a':s1, 'b':s2}))
</pre></td>
<td><img width="90%" src="./figures/join/join3-12.png"></td>
<td>辞書で与えると、辞書のkeyがインデックスとして採用されマルチインデックスとなる。</td></tr>    
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-13</td>
<td>
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
print(pd.concat([s1, s2], keys=['a', 'b']))
</pre></td>
<td><img width="90%" src="./figures/join/join3-12.png"></td>
<td>keys=を用いても同じことができる。</td></tr>    
 
</tbody>
</table>

In [7]:
print('''
### 縦連結(concat axis=0)
# 入力データ　
>>> dt0 = pd.DataFrame({'a':[1, 2], 'b':[1, 2], 'c':[1, 2]})
>>> print(dt0)
''')
dt0 = pd.DataFrame({'a':[1 ,2], 'b':[1, 2], 'c':[1, 2]})
print(dt0)

print('''
>>> dt1 = pd.DataFrame({'a':[3, 4], 'b':[3, 4], 'c':[3, 4]})
>>> print(dt1)
''')
dt1 = pd.DataFrame({'a':[3, 4], 'b':[3, 4], 'c':[3, 4]})
print(dt1)

print('''
>>> dt2 = pd.DataFrame({'b':[3, 4], 'c':[3, 4], 'd':[3, 4]})
>>> print(dt2)
''')
dt2 = pd.DataFrame({'b':[3, 4], 'c':[3, 4], 'd':[3, 4]})
print(dt2)

print('''
# no.3-1
# axis=0を省略すると縦連結
>>> print(pd.concat([dt0, dt1]))''')
print(pd.concat([dt0, dt1]))

print('''
# no.3-2
# 一つずつ連結するならappendを使っても同じ
>>> print(dt0.append(dt1))''')
print(dt0.append(dt1))

print('''
# no.3-3
# ignore_index=Trueとすればindexはリセットされる
>>> print(pd.concat([dt0, dt1], ignore_index=True))''')
print(pd.concat([dt0, dt1], ignore_index=True))

print('''
# no. 3-4
# 列名が一部異なるDataFrameの連結
# join='outer'がデフォルト
>>> print(pd.concat([dt0, dt2]))''')
print(pd.concat([dt0, dt2]))

print('''
# no.3-5 
# appendはouterのみで動作する。
>>> print(dt0.append(dt2))''')
print(dt0.append(dt2))

print('''
# no.3-6 
# inner連結
>>> pd.concat([dt0, dt2], join='inner')''')
print(pd.concat([dt0, dt2], join='inner'))

print('''
### 横連結(concat axis=1)
# 入力データ　
>>> dc0 = pd.DataFrame({'a':['a', 'a', 'a'], 'b':['b', 'b', 'b']}, index=[1, 2, 3])
>>> print(dc0)''')
dc0 = pd.DataFrame({'a':['a', 'a', 'a'], 'b':['b', 'b', 'b']}, index=[1, 2, 3])
print(dc0)

print('''
>>> dc1 = pd.DataFrame({'c':['c', 'c', 'c'], 'd':['d', 'd', 'd']}, index=[1, 2, 3])
>>> print(dc1)''')
dc1 = pd.DataFrame({'c':['c', 'c', 'c'], 'd':['d', 'd', 'd']}, index=[1, 2, 3])
print(dc1)

print('''
>>> dc2 = pd.DataFrame({'c':['c', 'c', 'c'], 'd':['d', 'd', 'd']}, index=[2, 3, 4])
>>> print(dc2)''')
dc2 = pd.DataFrame({'c':['c', 'c', 'c'], 'd':['d', 'd', 'd']}, index=[2, 3, 4])
print(dc2)

print('''
# no.3-7
>>> print(pd.concat([dc0, dc1], axis=1))''')
print(pd.concat([dc0, dc1], axis=1))

print('''
# no.3-8
# join='outer'がデフォルト
>>> print(pd.concat([dc0, dc2], axis=1))''')
print(pd.concat([dc0, dc2], axis=1))

print('''
# no.3-9
# inner連結
>>> print(pd.concat([dc0, dc2], axis=1, join='inner'))''')
print(pd.concat([dc0, dc2], axis=1, join='inner'))


print('''
### Seriesの連結
# 入力データ
>>> s1 = pd.Series([1, 2, 3, 4], index=[0, 1, 2, 3], name='s1')
>>> print(s1)''')
s1 = pd.Series([1, 2, 3, 4], index=[0, 1, 2, 3], name='s1')
print(s1)

print('''
>>> s2 = pd.Series([1, 2, 3, 4], index=[3, 1, 0, 4], name='s2')
>>> print(s2)''')
s2 = pd.Series([1, 2, 3, 4], index=[3, 1, 0, 4], name='s2')
print(s2)

print('''
# Seriesの縦連結。
# nameが異なるとNoneにセットされ、同じであれば受け継がれる。
# 結果もSeriesとなる。
# no.3-10
>>> print(pd.concat([s1, s2]))''')
print(pd.concat([s1, s2]))

print('''
# no.3-11
# axis=1で横連結すると結果はDataFrameになる。
>>> print(pd.concat([s1, s2], axis=1))''')
print(pd.concat([s1, s2], axis=1))

print('''
# no.3-12
# 辞書で与えると、辞書のkeyがインデックスとして採用されマルチインデックスとなる。
>>> print(pd.concat({'a':s1, 'b':s2}))''')
print(pd.concat({'a':s1, 'b':s2}))
 
print('''
# no.3-13
# keys=を用いても同じことができる。
>>> print(pd.concat([s1, s2], keys=['a', 'b']))''')
print(pd.concat([s1, s2], keys=['a', 'b']))


### 縦連結(concat axis=0)
# 入力データ　
>>> dt0 = pd.DataFrame({'a':[1, 2], 'b':[1, 2], 'c':[1, 2]})
>>> print(dt0)

   a  b  c
0  1  1  1
1  2  2  2

>>> dt1 = pd.DataFrame({'a':[3, 4], 'b':[3, 4], 'c':[3, 4]})
>>> print(dt1)

   a  b  c
0  3  3  3
1  4  4  4

>>> dt2 = pd.DataFrame({'b':[3, 4], 'c':[3, 4], 'd':[3, 4]})
>>> print(dt2)

   b  c  d
0  3  3  3
1  4  4  4

# no.3-1
# axis=0を省略すると縦連結
>>> print(pd.concat([dt0, dt1]))
   a  b  c
0  1  1  1
1  2  2  2
0  3  3  3
1  4  4  4

# no.3-2
# 一つずつ連結するならappendを使っても同じ
>>> print(dt0.append(dt1))
   a  b  c
0  1  1  1
1  2  2  2
0  3  3  3
1  4  4  4

# no.3-3
# ignore_index=Trueとすればindexはリセットされる
>>> print(pd.concat([dt0, dt1], ignore_index=True))
   a  b  c
0  1  1  1
1  2  2  2
2  3  3  3
3  4  4  4

# no. 3-4
# 列名が一部異なるDataFrameの連結
# join='outer'がデフォルト
>>> print(pd.concat([dt0, dt2]))
     a  b  c    d
0  1.0  1  1  NaN
1  2.0  2  2  NaN
0  NaN  3  3  3.0
1  NaN  4  4  4.0

# no.3-5 
# appendはouterのみで動作する。
>>> print(dt0.append(dt2))
     a  